In [57]:
import pandas as pd

In [58]:
df = pd.read_csv(r'D:\Tech-Challenge-QuintoAndar\database\data\quintoandar_santana_20250919.csv')

In [59]:
df.head()

,url,status,title,address_street,area,quartos,suite,banheiros,vagas,andar,pet,mobiliado,metro_proximo,Aluguel,Condomínio,IPTU,Seguro incêndio,Taxa de serviço,Total
0,https://www.quintoandar.com.br/imovel/89488144...,success,"Apartamento para alugar com 27m², 1 quarto e s...",Condomínios em São Paulo,27.0,1.0,0,1.0,NaN,4° a 7° andar,sim,não,sim,R$ 2.300,R$ 300,R$ 83,R$ 30,R$ 59,R$ 2.772
1,https://www.quintoandar.com.br/imovel/89510751...,success,"Apartamento para alugar com 52m², 2 quartos e ...",Condomínios em São Paulo,52.0,2.0,0,1.0,1.0,Até 3° andar,sim,não,sim,R$ 1.910,R$ 700,R$ 0,R$ 25,R$ 49,R$ 2.684
2,https://www.quintoandar.com.br/imovel/89501526...,success,"Apartamento para alugar com 35m², 1 quarto e 1...",Condomínios em São Paulo,35.0,1.0,0,1.0,1.0,Até 3° andar,não,não,sim,R$ 1.300,R$ 765,R$ 0,R$ 17,R$ 33,R$ 2.115
3,https://www.quintoandar.com.br/imovel/89468415...,success,"Apartamento para alugar com 12m², 1 quarto e s...",Condomínios em São Paulo,12.0,1.0,1,1.0,NaN,Até 3° andar,sim,não,sim,R$ 1.149,R$ 20,R$ 53,R$ 15,R$ 29,R$ 1.266
4,https://www.quintoandar.com.br/imovel/89499925...,success,"Apartamento para alugar com 70m², 2 quartos e ...",Condomínios em São Paulo,70.0,2.0,0,1.0,NaN,Até 3° andar,sim,não,não,R$ 2.500,R$ 350,R$ 0,R$ 32,R$ 64,R$ 2.946


In [60]:
df['url'].iloc[1]

'https://www.quintoandar.com.br/imovel/895107518/alugar/apartamento-2-quartos-santana-sao-paulo?from_route=%22search_results%22&house_tags=exclusivity&house_tags=newAd&house_tags=rentPriceDecreased&search_id=%22c9d4d38d-3325-473f-a2cd-029fcbfb212b%22&search_rank=%7B%22sortMode%22%3A%22relevance%22%2C%22searchMode%22%3A%22list%22%2C%22resultsOrigin%22%3A%22search%22%2C%22rank%22%3A0%2C%22personalization%22%3Atrue%7D'